In [44]:
import os
import pandas as pd 
import joblib
import glob
from copy import deepcopy
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_pinball_loss

In [45]:
BASE_PATH = os.getcwd()

In [46]:
model_paths = glob.glob(BASE_PATH + "/models/i5_models/*.pkl")
models = {model_path: joblib.load(model_path) for model_path in model_paths}

In [47]:
data = pd.read_csv("/Users/florian/Documents/github/DP2/Energy_production_price_prediction/Generation_forecast/Solar_forecast/data/train_norm.csv")

In [48]:
df = deepcopy(data)

X = df.drop(columns="Target_Capacity_MWP_%")
y = df["Target_Capacity_MWP_%"]


_, X_test, _, y_test = train_test_split(X, y, test_size=0.2, random_state=0, shuffle=False)

In [49]:
new_models = {}

for model_path, model in models.items():
    model.fit(X_test, y_test)
    new_models[model_path.split("/")[-1]] = model

In [50]:
for model_name, model in new_models.items():
    os.makedirs(BASE_PATH + "/models/i7_models_5retrain/", exist_ok=True)
    joblib.dump(model, BASE_PATH + "/models/i7_models_5retrain/" + f"{model_name}")

In [51]:
api_data = pd.read_csv("/Users/florian/Documents/github/DP2/Energy_production_price_prediction/Generation_forecast/Solar_forecast/data/test_norm.csv")
api_df = deepcopy(api_data)

In [52]:
X = api_df.drop(columns="Target_Capacity_MWP_%")
y = api_df["Target_Capacity_MWP_%"]

In [53]:
test_mean_max_capacity = 2778.9489032132205 # Hardcoded value from the loading api data

In [54]:
api_predictions = {model_name: model.predict(X) for model_name, model in new_models.items()}
alpha_vals = {"q1": 0.1, "q2": 0.2, "q3": 0.3, "q4": 0.4, "q5": 0.5, "q6": 0.6, "q7": 0.7, "q8": 0.8, "q9": 0.9}
api_mean_pinball_loss = {model_name: mean_pinball_loss(y, api_predictions[model_name], alpha= alpha_vals[model_name.split("_")[-1].split(".")[0]]) for model_name in api_predictions}

In [55]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(y=y, name="True Values", mode="lines", line=dict(color="black")))
for model_name, prediction in api_predictions.items():
    fig.add_trace(go.Scatter(y=prediction, name=f"{model_name.split("_")[-1].split(".")[0]}, Loss: {api_mean_pinball_loss[model_name]*test_mean_max_capacity}", mode="lines"))
fig.update_layout(title="Predictions vs True Values from LGBR model iter 5 retrained", xaxis_title="Time", yaxis_title="Solar Production MWh %")
fig.show()